In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
plt.style.use('ggplot')
%matplotlib inline

# 训练集特征说明
+ PassengerID (ID)
2. Survived (是否存活)
3. Pclass (客舱等级，重要)
4. Name (姓名，可结合爬虫)
5. Sex (性别，重要)
6. Age (年龄，重要)
7. SibSp (旁系亲友)
8. Parch (直系亲属)
9. Ticket (票编号)
10. Fare (票价)
11. Cabin (客舱编号)
12. Embarked (上传港口编号)

In [188]:
train_data = pd.read_csv('Titanic Data/train_local.csv')
train_data.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,1,1,"Dodge, Master. Washington",male,4.0,0,2,33638,81.8583,A34,S
1,2,0,3,"Mitkoff, Mr. Mito",male,NaN,0,0,349221,7.8958,NaN,S
2,3,1,3,"Johnson, Miss. Eleanor Ileen",female,1.0,1,1,347742,11.1333,NaN,S
3,4,0,2,"West, Mr. Edwy Arthur",male,36.0,1,2,C.A. 34651,27.7500,NaN,S
4,5,0,2,"Hart, Mr. Benjamin",male,43.0,1,1,F.C.C. 13529,26.2500,NaN,S


可以看到有三个特征属性有缺失

In [189]:
train_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 623 entries, 0 to 622
Data columns (total 12 columns):
PassengerId    623 non-null int64
Survived       623 non-null int64
Pclass         623 non-null int64
Name           623 non-null object
Sex            623 non-null object
Age            499 non-null float64
SibSp          623 non-null int64
Parch          623 non-null int64
Ticket         623 non-null object
Fare           623 non-null float64
Cabin          139 non-null object
Embarked       620 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 58.5+ KB


In [190]:
test_data = pd.read_csv('Titanic Data/test_local.csv')
test_data.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,624,3,"Moubarek, Master. Halim Gonios (""William George"")",male,NaN,1,1,2661,15.2458,NaN,C
1,625,2,"Kvillner, Mr. Johan Henrik Johannesson",male,31.0,0,0,C.A. 18723,10.5000,NaN,S
2,626,3,"Alhomaki, Mr. Ilmari Rudolf",male,20.0,0,0,SOTON/O2 3101287,7.9250,NaN,S
3,627,2,"Harper, Miss. Annie Jessie ""Nina""",female,6.0,0,1,248727,33.0000,NaN,S
4,628,3,"Nicola-Yarred, Miss. Jamila",female,14.0,1,0,2651,11.2417,NaN,C


测试集有三个缺失

In [191]:
test_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 268 entries, 0 to 267
Data columns (total 11 columns):
PassengerId    268 non-null int64
Pclass         268 non-null int64
Name           268 non-null object
Sex            268 non-null object
Age            215 non-null float64
SibSp          268 non-null int64
Parch          268 non-null int64
Ticket         268 non-null object
Fare           268 non-null float64
Cabin          65 non-null object
Embarked       267 non-null object
dtypes: float64(2), int64(4), object(5)
memory usage: 23.2+ KB


In [192]:
all_data = pd.concat([train_data,test_data],ignore_index=True,sort=False)

将测试集和训练集合并，做统一预处理

In [193]:
all_data

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,1.0,1,"Dodge, Master. Washington",male,4.0,0,2,33638,81.8583,A34,S
1,2,0.0,3,"Mitkoff, Mr. Mito",male,NaN,0,0,349221,7.8958,NaN,S
2,3,1.0,3,"Johnson, Miss. Eleanor Ileen",female,1.0,1,1,347742,11.1333,NaN,S
3,4,0.0,2,"West, Mr. Edwy Arthur",male,36.0,1,2,C.A. 34651,27.7500,NaN,S
4,5,0.0,2,"Hart, Mr. Benjamin",male,43.0,1,1,F.C.C. 13529,26.2500,NaN,S
...,...,...,...,...,...,...,...,...,...,...,...,...
886,887,NaN,3,"Lulic, Mr. Nikola",male,27.0,0,0,315098,8.6625,NaN,S
887,888,NaN,1,"Parr, Mr. William Henry Marsh",male,NaN,0,0,112052,0.0000,NaN,S
888,889,NaN,1,"Millet, Mr. Francis Davis",male,65.0,0,0,13509,26.5500,E38,S
889,890,NaN,3,"Calic, Mr. Petar",male,17.0,0,0,315086,8.6625,NaN,S


In [194]:
all_data['Survived'].value_counts()

0.0    392
1.0    231
Name: Survived, dtype: int64

- 不同称呼的乘客幸存率不同(把name中的称呼都提取出来，并且对同个人具有多个称呼标签进行统一）

In [195]:
all_data['Name'].head()

0       Dodge, Master. Washington
1               Mitkoff, Mr. Mito
2    Johnson, Miss. Eleanor Ileen
3           West, Mr. Edwy Arthur
4              Hart, Mr. Benjamin
Name: Name, dtype: object

In [196]:
all_data['Title'] = all_data['Name'].apply(lambda x:x.split(',')[1].split('.')[0].strip())
Title_dict = {}
Title_dict.update(dict.fromkeys(['Capt','Col','Major','Dr','Rev'],'Officer'))
Title_dict.update(dict.fromkeys(['Don','Str','the Countess','Dona','Lady'],'Royalty'))
Title_dict.update(dict.fromkeys(['Mme','Ms','Mrs'],'Mrs'))
Title_dict.update(dict.fromkeys(['Mlle','Miss'],'Miss'))
Title_dict.update(dict.fromkeys(['Mr'],'Mr'))
Title_dict.update(dict.fromkeys(['Master','Jonkheer'],'Master'))
all_data['Title'] = all_data['Title'].map(Title_dict)

In [197]:
all_data['Title']

0      Master
1          Mr
2        Miss
3          Mr
4          Mr
        ...  
886        Mr
887        Mr
888        Mr
889        Mr
890        Mr
Name: Title, Length: 891, dtype: object

- 将直系亲属和旁系亲属合并，并根据数量划分成三个区间（Label）

In [198]:
# 亲属数量
all_data['FamilySize'] = all_data['SibSp'] + all_data['Parch'] + 1
def Fam_label(s):
    if ( s>= 1) and (s <= 4):
        return 2
    elif ((s>4) and (s<=7))or (s == 1):
        return 1
    elif s > 7:
        return 0
all_data['FamilyLabel'] = all_data['FamilySize'].apply(Fam_label)

- 不同客舱的乘客幸存率不同 （新增Deck特征，先把Cabin空缺值填充为Unknown，再提取Cabin中的首字母构成乘客的甲板号）

In [199]:
all_data['Cabin'] = all_data['Cabin'].fillna('Unknown')
all_data['Desk'] = all_data['Cabin'].str.get(0)

- 共票数与生存率的关系（统计每个乘客的共票数）

In [200]:
Ticket_count = dict(all_data['Ticket'].value_counts())
all_data['TicketGroup'] = all_data['Ticket'].apply(lambda x:Ticket_count[x])
def Ticket_depart(s):
    if (s >= 2) and (s <= 4):
        return 2
    elif ((s>4) and (s<=8)) or (s == 1):
        return 1
    elif s > 8:
        return 0
all_data['TicketGroup'] = all_data['TicketGroup'].apply(Ticket_depart)

# 数据清洗
1. 缺失值清洗

Age缺失值用Sex,Title,Pclass三个特征构建随机森林模型，填充年龄缺失值

In [201]:
from sklearn.ensemble import RandomForestRegressor
age_df = all_data[['Age','Pclass','Sex','Title']]
age_df = pd.get_dummies(age_df)
#只会将分类变量变成虚拟变量，不会将连续的数值变量变成虚拟变量
age_df

,Age,Pclass,Sex_female,Sex_male,Title_Master,Title_Miss,Title_Mr,Title_Mrs,Title_Officer,Title_Royalty
0,4.0,1,0,1,1,0,0,0,0,0
1,NaN,3,0,1,0,0,1,0,0,0
2,1.0,3,1,0,0,1,0,0,0,0
3,36.0,2,0,1,0,0,1,0,0,0
4,43.0,2,0,1,0,0,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...
886,27.0,3,0,1,0,0,1,0,0,0
887,NaN,1,0,1,0,0,1,0,0,0
888,65.0,1,0,1,0,0,1,0,0,0
889,17.0,3,0,1,0,0,1,0,0,0


In [202]:
known_age = np.asarray(age_df[age_df.Age.notnull()])
unknown_age = np.asarray(age_df[age_df.Age.isnull()])
y = known_age[:,0]
X = known_age[:,1:]
rfr = RandomForestRegressor(random_state=0,n_estimators=100,n_jobs=-1)
rfr.fit(X,y)
predictedAge = rfr.predict(unknown_age[:,1:])
all_data.loc[(all_data.Age.isnull()) ,'Age'] = predictedAge

In [203]:
predictedAge[:10]

array([28.66816391, 32.74627389, 40.49831745, 16.00323396, 28.66816391,
       33.48576083, 28.66816391, 41.50062012, 32.74627389, 41.50062012])

embarked 也有2个缺失值，根据这个个体的其他信息，直接填充

In [204]:
all_data[all_data['Embarked'].isnull()]

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Title,FamilySize,FamilyLabel,Desk,TicketGroup
32,33,1.0,3,"Goldsmith, Mrs. Frank John (Emily Alice Brown)",female,31.0,1,1,363291,20.525,Unknown,NaN,Mrs,3,2,U,2
55,56,0.0,3,"Nosworthy, Mr. Richard Cater",male,21.0,0,0,A/4. 39886,7.800,Unknown,NaN,Mr,1,2,U,1
68,69,1.0,1,"Icard, Miss. Amelie",female,38.0,0,0,113572,80.000,B28,NaN,Miss,1,2,B,2
859,860,NaN,1,"Stone, Mrs. George Nelson (Martha Evelyn)",female,62.0,0,0,113572,80.000,B28,NaN,Mrs,1,2,B,2


In [205]:
all_data.groupby(by=['Pclass','Embarked']).Fare.median()

Pclass  Embarked
1       C           78.2667
        Q           90.0000
        S           52.0000
2       C           24.0000
        Q           12.3500
        S           13.5000
3       C            7.8958
        Q            7.7500
        S            8.0500
Name: Fare, dtype: float64

In [206]:
all_data['Embarked'] = all_data['Embarked'].fillna('C')

fare 的缺失值，用中位数填充

In [207]:
all_data[all_data['Fare'].isnull()]

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Title,FamilySize,FamilyLabel,Desk,TicketGroup


没有？那没事了

In [208]:
#fare = all_data[(all_data['Embarked'] == 'S') & (all_data['Pclass'] == 3)].Fare.median()
#all_data['Fare'] = all_data['Fare'].fillna(fare)

2. 异常值处理

- 多人家庭中没有获救的女性和儿童为异常值

In [209]:
all_data['Surname'] = all_data['Name'].apply(lambda x : x.split(',')[0].strip()) 
Surname_count = dict(all_data['Surname'].value_counts())
all_data['Family_count'] = all_data['Surname'].apply(lambda x : Surname_count[x])

#找出所有处于多人家庭的儿童和妇女和成年男性
Female_Child_Count = all_data.loc[(all_data['Family_count'] >= 2) & ((all_data['Age'] <= 12) | (all_data['Sex'] == 'female'))]
Male_Adult_Count = all_data.loc[(all_data['Family_count'] >= 2) & (all_data['Age'] > 12) & (all_data['Sex'] == 'male')]

Female_Child_Count

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Title,FamilySize,FamilyLabel,Desk,TicketGroup,Surname,Family_count
2,3,1.0,3,"Johnson, Miss. Eleanor Ileen",female,1.000000,1,1,347742,11.1333,Unknown,S,Miss,3,2,U,2,Johnson,6
6,7,1.0,2,"Collyer, Mrs. Harvey (Charlotte Annie Tate)",female,31.000000,1,1,C.A. 31921,26.2500,Unknown,S,Mrs,3,2,U,2,Collyer,3
10,11,1.0,1,"Thayer, Mrs. John Borland (Marian Longstreth M...",female,39.000000,1,1,17421,110.8833,C68,C,Mrs,3,2,C,2,Thayer,3
14,15,1.0,1,"Frauenthal, Mrs. Henry William (Clara Heinshei...",female,40.498317,1,0,PC 17611,133.6500,Unknown,S,Mrs,2,2,U,2,Frauenthal,2
17,18,0.0,3,"Panula, Master. Eino Viljami",male,1.000000,4,1,3101295,39.6875,Unknown,S,Master,6,1,U,1,Panula,6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
850,851,NaN,1,"Allison, Master. Hudson Trevor",male,0.920000,1,2,113781,151.5500,C22 C26,S,Master,4,2,C,2,Allison,3
851,852,NaN,2,"Kantor, Mrs. Sinai (Miriam Sternin)",female,24.000000,1,0,244367,26.0000,Unknown,S,Mrs,2,2,U,2,Kantor,2
854,855,NaN,1,"Ryerson, Miss. Susan Parker ""Suzette""",female,21.000000,2,2,PC 17608,262.3750,B57 B59 B63 B66,C,Miss,5,1,B,2,Ryerson,2
866,867,NaN,3,"Oreskovic, Miss. Marija",female,20.000000,0,0,315096,8.6625,Unknown,S,Miss,1,2,U,1,Oreskovic,2


In [210]:
Female_Child = pd.DataFrame(Female_Child_Count.groupby('Surname')['Survived'].mean().value_counts())
Female_Child.columns = ['GroupCount']
Female_Child

,GroupCount
1.000000,64
0.000000,22
0.666667,1
0.250000,1


In [211]:
Male_Adult = pd.DataFrame(Male_Adult_Count.groupby('Surname')['Survived'].mean().value_counts())
Male_Adult.columns = ['GroupCount']
Male_Adult

,GroupCount
0.000000,67
1.000000,11
0.500000,4
0.333333,2


普遍规律是女性和儿童的幸存率较高， 成年男性的幸存率较低，所以把不符合普遍规律的反常组选出来单独处理。
把女性和儿童组里幸存率为0的组设置为遇难组，成年男性组里存活率为1的组设为幸存组，推测处于遇难组的女性和儿童幸存的可能性可能较低，处于幸存组的成年男性幸存的可能性较高。

In [212]:
Female_Child_Count = Female_Child_Count.groupby('Surname')['Survived'].mean()
Dead_List = set(Female_Child_Count[Female_Child_Count.apply(lambda x : x==0)].index)

Male_Adult_Count = Male_Adult_Count.groupby('Surname')['Survived'].mean()
Survived_List = set(Male_Adult_Count[Male_Adult_Count.apply(lambda x : x== 1)].index)

#将测试集中所有幸存组的成员改成女性和儿童，将遇难组的都改成男性
train = all_data.loc[all_data['Survived'].notnull()]
test = all_data.loc[all_data['Survived'].isnull()]
test.loc[(test['Surname'].apply(lambda x : x in Dead_List)),'Sex'] = 'male'
test.loc[(test['Surname'].apply(lambda x : x in Dead_List)),'Age'] = 60
test.loc[(test['Surname'].apply(lambda x : x in Dead_List)),'Title'] = 'Mr'
test.loc[(test['Surname'].apply(lambda x : x in Survived_List)),'Age'] = 5
test.loc[(test['Surname'].apply(lambda x : x in Survived_List)),'Sex'] = 'female'
test.loc[(test['Surname'].apply(lambda x : x in Survived_List)),'Title'] = 'Miss'

# 特征转换

In [213]:
all_data

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Title,FamilySize,FamilyLabel,Desk,TicketGroup,Surname,Family_count
0,1,1.0,1,"Dodge, Master. Washington",male,4.000000,0,2,33638,81.8583,A34,S,Master,3,2,A,1,Dodge,1
1,2,0.0,3,"Mitkoff, Mr. Mito",male,28.668164,0,0,349221,7.8958,Unknown,S,Mr,1,2,U,1,Mitkoff,1
2,3,1.0,3,"Johnson, Miss. Eleanor Ileen",female,1.000000,1,1,347742,11.1333,Unknown,S,Miss,3,2,U,2,Johnson,6
3,4,0.0,2,"West, Mr. Edwy Arthur",male,36.000000,1,2,C.A. 34651,27.7500,Unknown,S,Mr,4,2,U,2,West,3
4,5,0.0,2,"Hart, Mr. Benjamin",male,43.000000,1,1,F.C.C. 13529,26.2500,Unknown,S,Mr,3,2,U,2,Hart,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
886,887,NaN,3,"Lulic, Mr. Nikola",male,27.000000,0,0,315098,8.6625,Unknown,S,Mr,1,2,U,1,Lulic,1
887,888,NaN,1,"Parr, Mr. William Henry Marsh",male,41.500620,0,0,112052,0.0000,Unknown,S,Mr,1,2,U,1,Parr,1
888,889,NaN,1,"Millet, Mr. Francis Davis",male,65.000000,0,0,13509,26.5500,E38,S,Mr,1,2,E,1,Millet,1
889,890,NaN,3,"Calic, Mr. Petar",male,17.000000,0,0,315086,8.6625,Unknown,S,Mr,1,2,U,1,Calic,2


In [274]:
train = all_data.loc[all_data['Survived'].notnull()]
test = all_data.loc[all_data['Survived'].isnull()]
all_data = pd.concat([train,test])
all_data = all_data[['Survived','Pclass','Sex','Age','Fare','Embarked','Title','FamilyLabel','Desk','TicketGroup']]
all_data = pd.get_dummies(all_data)
train = all_data[all_data['Survived'].notnull()]
test = all_data[all_data['Survived'].isnull()].drop('Survived',axis = 1)
X = np.array(train)[:,1:]
y = np.array(train)[:,0]

In [215]:
train

,Survived,Pclass,Age,Fare,FamilyLabel,TicketGroup,Sex_female,Sex_male,Embarked_C,Embarked_Q,...,Title_Royalty,Desk_A,Desk_B,Desk_C,Desk_D,Desk_E,Desk_F,Desk_G,Desk_T,Desk_U
0,1.0,1,4.000000,81.8583,2,1,0,1,0,0,...,0,1,0,0,0,0,0,0,0,0
1,0.0,3,28.668164,7.8958,2,1,0,1,0,0,...,0,0,0,0,0,0,0,0,0,1
2,1.0,3,1.000000,11.1333,2,2,1,0,0,0,...,0,0,0,0,0,0,0,0,0,1
3,0.0,2,36.000000,27.7500,2,2,0,1,0,0,...,0,0,0,0,0,0,0,0,0,1
4,0.0,2,43.000000,26.2500,2,2,0,1,0,0,...,0,0,0,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
618,1.0,3,21.000000,7.6500,2,1,1,0,0,0,...,0,0,0,0,0,0,0,0,0,1
619,0.0,1,41.500620,31.0000,2,2,0,1,0,0,...,0,0,0,0,0,0,0,0,0,1
620,0.0,3,41.000000,14.1083,2,1,0,1,0,0,...,0,0,0,0,0,0,0,0,0,1
621,1.0,1,14.000000,120.0000,2,2,1,0,0,0,...,0,0,1,0,0,0,0,0,0,0


In [219]:
X = np.array(train)[:,1:]
y = np.array(train)[:,0]

In [140]:
X = X/X.max(axis=0)

In [71]:
X[121]

array([ 3.        , 16.00323396,  7.75      ,  2.        ,  1.        ,
        1.        ,  0.        ,  0.        ,  1.        ,  0.        ,
        0.        ,  1.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  1.        ])

In [67]:
X.shape,y.shape

((623, 25), (623,))

In [68]:
from sklearn.pipeline import Pipeline
from sklearn.ensemble import  RandomForestClassifier
from sklearn.model_selection import  GridSearchCV
from sklearn.feature_selection import SelectKBest
from warnings import simplefilter
simplefilter(action='ignore', category=FutureWarning)

In [70]:
import warnings
warnings.filterwarnings("ignore")
X = np.array(train)[:,1:]
y = np.array(train)[:,0]
pipe = Pipeline([('select', SelectKBest(k=20)),
            ('classify',RandomForestClassifier(random_state=10,max_features='sqrt'))
            ])
parm_test = {'classify__n_estimators':list(range(20,50,2)),
            'classify__max_depth':list(range(3,60,3))
            }

gsearch = GridSearchCV(estimator=pipe,param_grid=parm_test,scoring='roc_auc',cv = 10)
gsearch.fit(X,y)
print(gsearch.best_params_,gsearch.best_score_)

{'classify__max_depth': 9, 'classify__n_estimators': 34} 0.8685520980403735


In [33]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim 

In [166]:
class LR(nn.Module):
    def __init__(self):
        super(LR,self).__init__()
        self.inchannel = 25
        #self.fc1 = nn.Linear(25,128) # 由于25个维度已经固定了，所以这里写25
        #self.fc2 = nn.Linear(128,256)
        #self.fc3 = nn.Linear(256,512)
        #self.fc4 = nn.Linear(512,2)
        self.layer0 = self.make_layer(ResidualBlock, 64)
        self.layer1 = self.make_layer(ResidualBlock, 128)
        self.layer2 = self.make_layer(ResidualBlock, 256)
        #self.layer3 = self.make_layer(ResidualBlock, 512)
        self.layer4 = nn.Linear(256,2)
    def make_layer(self, block, channels):
        layers = []
        layers.append(block(self.inchannel, channels))
        self.inchannel = channels
        return nn.Sequential(*layers)
    
    def forward(self,x):
        #block1 = self.fc1(x)
        #block1 = F.relu(block1)
        #block1 = F.dropout(block1,p=0.5)
        
        #block2 = self.fc2(block1)
        #block2 = F.relu(block2)
        #block2 = F.dropout(block2,p=0.5)

        #block3 = self.fc3(block2)
        #block3 = F.relu(block3)
        #block3 = F.dropout(block3,p=0.5)
        
        #out = self.fc4(block3)
        out = self.layer0(x)
        out = self.layer1(out)
        out = self.layer2(out)
        #out = self.layer3(out)
        out = self.layer4(out)
        out = torch.sigmoid(out)
        return out

In [142]:
class ResidualBlock(nn.Module):
    def __init__(self, inchannel, outchannel):
        super(ResidualBlock, self).__init__()
        self.left = nn.Sequential(
            nn.Linear(inchannel,outchannel),
            nn.ReLU(inplace=True),
            nn.Linear(outchannel,outchannel),
        )
        self.shortcut = nn.Sequential()
        if inchannel != outchannel:
            self.shortcut = nn.Sequential(
                nn.Linear(inchannel,outchannel),
            )

    def forward(self, x):
        out = self.left(x)
        out += self.shortcut(x)
        out = F.relu(out)
        return out

In [286]:
def testA(pred,lab):
    t=pred.max(-1)[1]==lab
    return torch.mean(t.float())

In [296]:
net=LR() 
criterion=nn.CrossEntropyLoss() # 使用CrossEntropyLoss损失
optm=torch.optim.Adam(net.parameters()) # Adam优化
epochs=1000 # 训练1000次

In [297]:
random_idx = np.arange(X.shape[0])
np.random.seed(19)
np.random.shuffle(random_idx)
train_size = 550
for i in range(epochs):
    # 指定模型为训练模式，计算梯度
    net.train()
    # 输入值都需要转化成torch的Tensor
    train_data=torch.from_numpy(X[random_idx[:train_size],:]).float()
    label_data=torch.from_numpy(y[random_idx[:train_size]]).long()
    y_hat=net(train_data)
    loss=criterion(y_hat,label_data) # 计算损失
    optm.zero_grad() # 前一步的损失清零
    loss.backward() # 反向传播
    optm.step() # 优化
    if (i+1)%100 ==0 : # 这里我们每100次输出相关的信息
        # 指定模型为计算模式
        net.eval()
        accu1 = testA(y_hat,label_data)
        
        test_in = torch.from_numpy(X[random_idx[train_size:],:]).float()
        test_l = torch.from_numpy(y[random_idx[train_size:]]).long()
        test_out = net(test_in)
        # 使用我们的测试函数计算准确率
        accu2 = testA(test_out,test_l)
        print("Epoch:{},Loss:{:.4f}，train_Accuracy：{:.2f},test_Accuracy2：{:.2f}".format(i+1,loss.item(),accu1,accu2))

Epoch:100,Loss:0.4618，train_Accuracy：0.85,test_Accuracy2：0.89
Epoch:200,Loss:0.4393，train_Accuracy：0.87,test_Accuracy2：0.88
Epoch:300,Loss:0.4248，train_Accuracy：0.88,test_Accuracy2：0.86
Epoch:400,Loss:0.4933，train_Accuracy：0.82,test_Accuracy2：0.86
Epoch:500,Loss:0.4280，train_Accuracy：0.88,test_Accuracy2：0.86
Epoch:600,Loss:0.4184，train_Accuracy：0.90,test_Accuracy2：0.85
Epoch:700,Loss:0.4190，train_Accuracy：0.90,test_Accuracy2：0.85
Epoch:800,Loss:0.4183，train_Accuracy：0.89,test_Accuracy2：0.85
Epoch:900,Loss:0.4162，train_Accuracy：0.90,test_Accuracy2：0.85
Epoch:1000,Loss:0.4161，train_Accuracy：0.90,test_Accuracy2：0.84


In [288]:
test_in=torch.from_numpy(np.array(test)).float()
test_out = net(test_in)

In [293]:
test_out_numpy = test_out.max(-1)[1].detach().numpy().astype('int32')

In [291]:
test

,Pclass,Age,Fare,FamilyLabel,TicketGroup,Sex_female,Sex_male,Embarked_C,Embarked_Q,Embarked_S,...,Title_Royalty,Desk_A,Desk_B,Desk_C,Desk_D,Desk_E,Desk_F,Desk_G,Desk_T,Desk_U
623,3,5.259208,15.2458,2,2,0,1,1,0,0,...,0,0,0,0,0,0,0,0,0,1
624,2,31.000000,10.5000,2,1,0,1,0,0,1,...,0,0,0,0,0,0,0,0,0,1
625,3,20.000000,7.9250,2,1,0,1,0,0,1,...,0,0,0,0,0,0,0,0,0,1
626,2,6.000000,33.0000,2,2,1,0,0,0,1,...,0,0,0,0,0,0,0,0,0,1
627,3,14.000000,11.2417,2,2,1,0,1,0,0,...,0,0,0,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
886,3,27.000000,8.6625,2,1,0,1,0,0,1,...,0,0,0,0,0,0,0,0,0,1
887,1,41.500620,0.0000,2,1,0,1,0,0,1,...,0,0,0,0,0,0,0,0,0,1
888,1,65.000000,26.5500,2,1,0,1,0,0,1,...,0,0,0,0,0,1,0,0,0,0
889,3,17.000000,8.6625,2,1,0,1,0,0,1,...,0,0,0,0,0,0,0,0,0,1


In [295]:
submission = pd.DataFrame({'PassengerID' : test_data.PassengerId , 'Survived': test_out_numpy.astype(np.int32)})
submission.to_csv(r'best_submission.csv',index = False)